In [ ]:
import requests

headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
}


model= "deepseek-r1:1.5b"
prompt= "Hi"
data = f'{{\n  "model": {model} ,\n  "prompt": {prompt}\n}}'
response = requests.post('http://localhost:11434/api/generate', headers=headers, data=data)

print(response)

In [ ]:
from pydantic import BaseModel, Field
from typing import List
from graphviz import Digraph
from openai import OpenAI
# import instructor
import base64
from io import BytesIO
from PIL import Image


def visualize_knowledge_graph(kg: KnowledgeGraph):
    dot = Digraph(comment="Knowledge Graph")

    # Add nodes
    for node in kg.nodes:
        dot.node(str(node.id), node.label, color=node.color)

    # Add edges
    for edge in kg.edges:
        dot.edge(str(edge.source), str(edge.target), label=edge.label, color=edge.color)

    # Render the graph
    dot.render("knowledge_graph.gv", view=True)





class Node(BaseModel, frozen=True):
    id: int
    label: str
    color: str


class Edge(BaseModel, frozen=True):
    source: int
    target: int
    label: str
    color: str = "black"


class KnowledgeGraph(BaseModel):
    nodes: List[Node] = Field(..., default_factory=list)
    edges: List[Edge] = Field(..., default_factory=list)


# Adds response_model to ChatCompletion
# Allows the return of Pydantic model rather than raw JSON
# client = instructor.from_openai(OpenAI())
import instructor
client = instructor.from_openai(OpenAI())

# Convert the annotated image to a PIL Image
pil_image = Image.fromarray(annotated_image)

# Save the image to a BytesIO object
buffered = BytesIO()
pil_image.save(buffered, format="JPEG")

# Encode the image to base64
img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")


def generate_graph(input) -> KnowledgeGraph:
    return client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": [
                {"type": "text", "text": "You are provided an image of a document page whose different sections are overlaid with boxes. The boxes are numbered. You are to return a graph structure which conveys how the different boxes are related to each other. The nodes should be the numbers only and nothing else."},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{img_str}",
                    }
                },]
            ,
            }
        ],
        response_model=KnowledgeGraph,
    )  # type: ignore


graph = generate_graph("Teach me about quantum mechanics")
visualize_knowledge_graph(graph)

